In [50]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

# Usar renderer para notebooks (más rápido que browser)
pio.renderers.default = "notebook"

In [2]:
def load_mock_data():
    # Datos del Funnel
    funnel_data = {
        'Etapa': ['Usuarios Totales', 'Intención de Registro', 'Registro Finalizado'],
        'Cantidad': [35215, 3715, 615]
    }
    
    # Evolución Temporal (1 al 7 de Enero 2026)
    dates = pd.date_range(start="2026-01-01", end="2026-01-07")
    evolution_data = pd.DataFrame({
        'Fecha': dates,
        'Intención de Registro': [510, 780, 650, 720, 810, 1020, 1080], # Mock de sesiones
        'Registro': [45, 68, 95, 82, 195, 120, 360]
    })
    
    # Datos por Dispositivo
    device_data = pd.DataFrame({
        'Dispositivo': ['Mobile', 'Desktop', 'Tablet', 'Smart TV'],
        'Intención': [1450, 1380, 180, 100],
        'Registro': [380, 240, 20, 5]
    })

    return funnel_data, evolution_data, device_data

funnel, evolution, devices = load_mock_data()


In [64]:
valores_reales = funnel['Cantidad']
etapas = funnel['Etapa']

# Texto personalizado: "35,215,664 (100%)"
textos = [f"{v:,}" for v in valores_reales]
x_log = np.log10(valores_reales)

fig = go.Figure(go.Funnel(
    y = etapas,
    x = x_log,
    text = textos,
    textinfo = "text+percent initial", # Muestra el valor real y % respecto al inicio
    textposition = "inside",
    insidetextanchor = "middle",
    marker = {
        # "color": ["#2450A6", "#1E43E6", "#1E88E6"],
        "color": ["#2450A6", "#1565C0", "#4E8ACF"],
        "line": {"width": 1, "color": "white"}
    },
    connector = {"fillcolor": "#A6C6ED", "line": {"width": 0}},
    hoverinfo = "y+text", # Al pasar el mouse muestra Etapa + Valor Real
))

fig.update_layout(
    title = {"text": "<b>Embudo de Conversión - Infobae</b><br><span style='font-size:12px'>Escala logarítmica aplicada para visibilidad</span>"},
    paper_bgcolor = "white",
    plot_bgcolor = "white",
    xaxis = {
        "showticklabels": False, # Escondemos los números del logaritmo (2, 4, 6, 8)
        "showgrid": False,
        "zeroline": False
    },
    margin = dict(l=200), # Espacio para que no se corten los nombres de las etapas
    height = 500
)

fig.show()